In [1]:
# Import required packages
import threading
import time
import re
import logging
import os

#from tkinter import ttk
#import tkinter as tk
#from tkinter import scrolledtext
#from tkinter import messagebox as msg
#import tkinter.font as tkfont

from IPython.display import display, Markdown, clear_output, Image
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import matplotlib.figure as figure
import matplotlib.animation as animation
import matplotlib.dates as mdates
from matplotlib.ticker import FormatStrFormatter
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np

# Custom Classes
from meerkat import Meerkat, Response

In [ ]:
# Global variables

# Set DEBUG to True to allow for verbose printing to the terminal
DEBUG = False

In [ ]:
# Meerkat instantiation
meerkat = Meerkat()

In [ ]:
# defining some widgets
text = widgets.Text(
       value='My Text',
       description='Title', )
calendar = widgets.DatePicker(
           description='Select Date')
slider = widgets.FloatSlider(
         value=1,
         min=0,
         max=10.0,
         step=0.1,)
menu = widgets.Dropdown(
       options=['red', 'blue', 'green'],
       value='red',
       description='Color:')
checkbox = widgets.Checkbox(
           description='Check to invert',)

In [ ]:
menu

In [2]:
from IPython.display import display
import ipywidgets as widgets


def clicked(arg):
    print("button has been clicked!")

button_download = widgets.Button(description = 'Test Button')   
button_download.on_click(clicked)
display(button_download)

Button(description='Test Button', style=ButtonStyle())

button has been clicked!
button has been clicked!
button has been clicked!
button has been clicked!
button has been clicked!
button has been clicked!
button has been clicked!
button has been clicked!
button has been clicked!


In [ ]:
# Class for instantiating instructions in the GUI
class Instructions(tk.Frame):
    def __init__(self, parent, master, tab):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.master = master

        self.frame = tk.LabelFrame(tab, height=100, bd=3, relief='groove', text="Instructions", fg="blue")
        self.frame.pack(padx=7, pady=7, fill=tk.BOTH)

        tk.Label(self.frame, text="\tStep 1: Select the serial port connected to the demo kit.").grid(sticky=tk.W)
        tk.Label(self.frame, text="\tStep 2: Click Connect in the Serial section.").grid(sticky=tk.W)
        tk.Label(self.frame, text="\tStep 3: Wait for Status to say ‘Connected’. "
                                  "Note: this may take up to one minute.").grid(sticky=tk.W)
        tk.Label(self.frame, text="\tStep 4: Click on the Safari Data tab and click on Start Demo.").grid(sticky=tk.W)

In [ ]:
# Class for Serial connectivity
class Serial(tk.Frame):
    def __init__(self, parent, master, tab):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.master = master

        self.frame = tk.LabelFrame(tab, height=100, bd=3, relief='groove', text="Serial Connection", fg="blue")
        self.frame.pack(padx=7, pady=7, fill=tk.BOTH)

        self.frame_instructions = tk.Frame(self.frame)
        self.frame_instructions.grid(row=0, column=0, ipady=7, sticky=tk.W)

        self.frame_status = tk.Frame(self.frame)
        self.frame_status.grid(row=1, column=0, sticky=tk.W)

        self.frame_serial = tk.Frame(self.frame)
        self.frame_serial.grid(row=2, column=0, pady=7, sticky=tk.W)
        self.frame_serial.columnconfigure(0, minsize=200)

        tk.Label(self.frame_instructions,
                 text="Select the desired serial port from the dropdown list").grid(row=1, column=0)
        tk.Label(self.frame_status, text="Status: ").grid(row=0, column=0, sticky=tk.E)
        self.label_serial_status = tk.Label(self.frame_status,
                                            text="Disconnected",
                                            fg="red")
        self.label_serial_status.grid(row=0, column=1, sticky=tk.W)

        self.serial_ports = meerkat.serial_port_scan()
        self.selectedPort = tk.StringVar()
        self.serialPortMenu = tk.OptionMenu(self.frame_serial, self.selectedPort, tuple(self.serial_ports))
        self.serialButton = tk.Button(self.frame_serial)
        self.refresh_serial()
        self.t1 = None

    # function for rescanning the serial ports and refreshing the drop-down list
    def refresh_serial(self):
        serial_port_list = meerkat.serial_port_scan()

        try:
            self.serialPortMenu = tk.OptionMenu(self.frame_serial, self.selectedPort, *serial_port_list)
            self.selectedPort.set(serial_port_list[0])
            self.serialPortMenu.grid(row=0, column=0, sticky=tk.E + tk.W)
            self.serialButton = tk.Button(self.frame_serial, text="Connect", command=self.connect_serial)
            self.serialButton.grid(row=0, column=1)
        except IndexError:
            self.selectedPort.set("")
            tk.Label(self.frame_serial, text="No COM ports detected. Try refreshing.").grid(row=0, column=0)
            self.serialButton = tk.Button(self.frame_serial, text="Refresh", command=self.refresh_serial)
            self.serialButton.grid(row=0, column=1)
        except Exception as e:
            logging.exception(e)
            self.selectedPort.set("")
            tk.Label(self.frame_serial, text="No COM ports detected. Try refreshing.").grid(row=0, column=0)
            self.serialButton = tk.Button(self.frame_serial, text="Refresh", command=self.refresh_serial)
            self.serialButton.grid(row=0, column=1)

    # Function for opening a serial connection on the specified port
    def connect_serial(self):
        port = self.selectedPort.get()

        ret = meerkat.serial_connect(port)

        if ret == 0:
            self.serialPortMenu.configure(state=tk.DISABLED)
            self.serialButton.configure(text="Disconnect", command=self.disconnect_serial)
            # Don't show connected until successful comms with Meerkat
            # self.label_serial_status.configure(text="Connected", fg="green")

            # Start a thread to handle receiving serial data
            self.t1 = threading.Thread(target=self.read_serial)
            self.t1.daemon = True
            self.t1.start()

            # Kill any running processes
            meerkat.serial_ctrl_c()
            time.sleep(0.25)
            meerkat.serial.write("root\n".encode('utf-8'))
        else:
            msg.showerror("Serial Port Error",
                          "Error Opening Serial Port. Please check serial connection and try again")

    def read_serial(self):
        # Infinite loop is okay since this is running in it's own thread.
        while True:
            c = meerkat.serial.read().decode('unicode_escape')  # attempt to read a character from Serial

            # was anything read?
            if len(c) == 0:
                break

            # check if character is a delimeter
            if c == '\r':
                c = ''  # don't want returns. chuck it

            if c == '\n':
                meerkat.serial_buffer += "\n"  # add the newline to the buffer

                # Parse the received serial data since we've received a full line
                self.parse_serial(meerkat.serial_buffer)

                # add the line to the TOP of the log
                if self.parent.developer_mode_enabled:
                    # Remove non-ansi characters such as text color modifiers
                    ansisequence = re.compile(r'\x1B\[[^A-Za-z]*[A-Za-z]')
                    modified_buffer = ansisequence.sub('', meerkat.serial_buffer)
                    self.parent.terminal.log.insert(tk.END, modified_buffer)
                    self.parent.terminal.log.see(tk.END)

                meerkat.serial_buffer = ""  # empty the buffer
            else:
                meerkat.serial_buffer += c  # add to the buffer

        self.master.after(10, self.read_serial)  # check serial again soon

    # Function for parsing the received serial string
    def parse_serial(self, serial_string):
        # Parsing the actual string is done by the Meerkat class.  We will parse the response from the Meerkat class.
        response = meerkat.serial_parse(serial_string)
        dfont = self.parent.font

        if response == Response.LOGIN:
            self.label_serial_status.configure(text="Connected", fg="green")
            self._login()
        elif response == Response.SETUP:
            self.label_serial_status.configure(text="Connected", fg="green")
            self.parent.device_setup()
        elif response == Response.IP_ADDRESS:
            self.parent.wifi.label_wifi_ip.configure(text=meerkat.ip_address)
        elif response == Response.WIFI_DISCONNECTED:
            self.parent.wifi.label_wifi_status.configure(text="Disconnected", fg="red")
            self.parent.wifi.label_wifi_ssid.configure(text="")
            self.parent.wifi.label_wifi_ip.configure(text="")
            self.parent.wifi.btn_wifi_change.grid_forget()
            self.parent.wifi.edit_wifi()
        elif response == Response.WIFI_CONNECTED:
            self.parent.wifi.label_wifi_status.configure(text="Connected", fg="green")
            strings = serial_string.split(":")
            ssid_string = strings[1].replace('"', "").rstrip()
            self.parent.wifi.label_wifi_ssid.configure(text=ssid_string)
            self.parent.wifi.btn_wifi_change.configure(text="Change", command=self.parent.wifi.edit_wifi)
            self.parent.wifi.btn_wifi_change.grid(row=3, column=0, sticky=tk.W + tk.E, columnspan=2)
            self.parent.wifi.frame_wifi_info.grid_forget()
            self.parent.wifi.frame_wifi_params.grid_forget()
        elif response == Response.WIFI_ERROR:
            msg.showerror("WiFi System Error", "Error connecting to WiFi, please reboot")
        elif response == Response.SAFARI_ERROR:
            meerkat.serial_ctrl_c()
            msg.showerror("AD7124 Error", "Please check your physical connections and restart the script.")
        elif response == Response.SAFARI_SERIAL_STREAMING:
            tk.Label(self.parent.data.frame_data,
                     text="Serial", font=dfont, bg="white", fg="green").grid(row=0, column=1, sticky=tk.W)
        elif response == Response.SAFARI_CLOUD_STREAMING:
            tk.Label(self.parent.data.frame_data,
                     text="Cloud", font=dfont, bg="white", fg="green").grid(row=0, column=1, sticky=tk.W)
        elif response == Response.SAFARI_DATA_RECEIVED:
            if meerkat.safari.test_mode_enabled:
                self.parent.test.update_test_data(serial_string)
            else:
                meerkat.update_plot_data(serial_string)
        elif response == Response.SAFARI_FAULT_DETECTED:
            print("Serial Fault Detected!")
            # Removing error box for customer demo to prevent excessive pop-ups and demo stopping.
            # msg.showerror("Safari Fault", "Accelerometer Fault Condition Detected. Please Check Motor Status.")

    def disconnect_serial(self):
        # This function is for disconnecting and quitting the application.
        # Sometimes the application throws a couple of errors while it is being shut down, the fix isn't out yet
        # but will be pushed to the repo once done.
        # simple GUI.quit() calls.

        try:
            meerkat.serial.close()
            self.serialButton.configure(text="Connect", command=self.connect_serial)
            self.serialPortMenu.configure(state=tk.NORMAL)
            self.label_serial_status.configure(text="Disconnected", fg="red")

            self.parent.wifi.label_wifi_status.configure(text="Disconnected", fg="red")
            self.parent.wifi.label_wifi_ssid.configure(text="")
            self.parent.wifi.label_wifi_ip.configure(text="")
            self.parent.wifi.btn_wifi_change.grid_forget()

        except AttributeError:
            print("Closed without Using it -_-")

    # Function for logging into the device
    def _login(self):
        if meerkat.serial_port_open():
            if DEBUG:
                print("Attempting to login to Meerkat")
            meerkat.login()
            self.parent.device_setup()


In [ ]:
# Class controlling WiFi

class Wifi(tk.Frame):
    def __init__(self, parent, master, tab):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.master = master

        self.frame = tk.LabelFrame(tab, height=100, bd=3, relief='groove', text="Meerkat Board WiFi Connection",
                                   fg="blue")
        self.frame.pack(padx=7, pady=7, fill=tk.BOTH)

        self.frame_wifi_status = tk.Frame(self.frame)
        self.frame_wifi_info = tk.Frame(self.frame)
        self.frame_wifi_params = tk.Frame(self.frame)
        self.frame_wifi_status.grid(sticky=tk.E + tk.W)

        tk.Label(self.frame_wifi_status, text="Status: ").grid(row=0, column=0, sticky=tk.E)
        self.label_wifi_status = tk.Label(self.frame_wifi_status, text="Disconnected", fg="red")
        self.label_wifi_status.grid(row=0, column=1, sticky=tk.W)

        tk.Label(self.frame_wifi_status, text="SSID: ").grid(row=1, column=0, sticky=tk.E)
        self.label_wifi_ssid = tk.Label(self.frame_wifi_status, text="")
        self.label_wifi_ssid.grid(row=1, column=1, sticky=tk.W)

        tk.Label(self.frame_wifi_status, text="IP Address: ").grid(row=2, column=0, sticky=tk.E)
        self.label_wifi_ip = tk.Label(self.frame_wifi_status, text="")
        self.label_wifi_ip.grid(row=2, column=1, sticky=tk.W)

        self.btn_wifi_change = tk.Button(self.frame_wifi_status, text="Change", command=self.edit_wifi)

        tk.Label(self.frame_wifi_info, text="").grid(row=0, column=0)
        tk.Label(self.frame_wifi_info, text="").grid(row=1, column=0)
        tk.Label(self.frame_wifi_info,
                 text="Enter the network credentials of the network you would "
                 "like Safari to connect to:").grid(row=2, column=0, columnspan=2)
        tk.Label(self.frame_wifi_info, text="NOTE: This only needs to be performed for new network connections",
                 fg="red").grid(row=3, column=0, sticky=tk.W, columnspan=2)
        tk.Label(self.frame_wifi_info, text="").grid(row=4, column=0)

        tk.Label(self.frame_wifi_params, text="SSID:").grid(row=3, column=0, sticky=tk.W)
        self.ssid_entry = tk.Entry(self.frame_wifi_params)
        self.ssid_entry.grid(row=3, column=1, columnspan=2, sticky=tk.W + tk.E)
        tk.Label(self.frame_wifi_params, text="Password:").grid(row=4, column=0, sticky=tk.W)
        self.password_entry = tk.Entry(self.frame_wifi_params)
        self.password_entry.grid(row=4, column=1, sticky=tk.W + tk.E)
        self.frame_wifi_params.columnconfigure(1, weight=1)

        self.btn_wifi_connect = tk.Button(self.frame_wifi_params, text="Connect", command=self.wifi_connect)
        self.btn_wifi_connect.grid(row=5, column=0, sticky=tk.W + tk.E, columnspan=2)

    # Function for connecting the device to Wi-Fi
    def wifi_connect(self):
        if meerkat.serial_port_open():
            meerkat.wifi_connect(self.ssid_entry.get(), self.password_entry.get())
            self.label_wifi_status.configure(text="Disconnected", fg="red")
            self.label_wifi_ssid.configure(text="")
            self.label_wifi_ip.configure(text="")
        else:
            msg.showerror("Serial Port Error", "Serial Port Not Open.")

    # Function for allowing the user to edit the Wi-Fi
    def edit_wifi(self):
        self.frame_wifi_info.grid(sticky=tk.E + tk.W)
        self.frame_wifi_params.grid(sticky=tk.E + tk.W)
        self.btn_wifi_change.configure(text="Cancel", command=self.edit_wifi_cancel)

    # Function for cancelling the request to edit the device Wi-Fi connection
    def edit_wifi_cancel(self):
        self.frame_wifi_info.grid_forget()
        self.frame_wifi_params.grid_forget()
        self.btn_wifi_change.configure(text="Change", command=self.edit_wifi)


In [ ]:
class Acn(tk.Frame):
    def __init__(self, parent, master, tab):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.master = master

        self.frame = tk.LabelFrame(tab, height=100, bd=3, relief='groove', text="User PC Connected to Arrow Connect",
                                   fg="blue")
        self.frame.pack(padx=7, pady=7, fill=tk.BOTH)

        tk.Label(self.frame, text="Status: ").grid(row=0, column=0, sticky=tk.E)
        self.label_acn_status_value = tk.Label(self.frame, text="Disconnected", fg="red")
        self.label_acn_status_value.grid(row=0, column=1, sticky=tk.W)

        tk.Label(self.frame, text="Device Name: ").grid(row=1, column=0, sticky=tk.E)
        self.label_acn_device_name_value = tk.Label(self.frame, text="")
        self.label_acn_device_name_value.grid(row=1, column=1, sticky=tk.W)

        tk.Label(self.frame, text="Device HID: ").grid(row=2, column=0, sticky=tk.E)
        self.label_acn_device_hid_value = tk.Label(self.frame, text="")
        self.label_acn_device_hid_value.grid(row=2, column=1, sticky=tk.W)

    # MQTT callback function
    def mqtt_callback(self, response):
        if response == Response.ACN_MQTT_CONNECTED:
            if DEBUG:
                print("MQTT Connected")
            self.parent.acn.label_acn_status_value.configure(text="Connected", fg="green")
            self.parent.acn.label_acn_device_name_value.configure(text=meerkat.acn.DEVICE_NAME)
            self.parent.acn.label_acn_device_hid_value.configure(text=meerkat.acn.DEVICE_HID)
        elif response == Response.ACN_MQTT_DISCONNECTED:
            if DEBUG:
                print("MQTT Disconnected")
            self.parent.acn.label_acn_status_value.configure(text="Disconnected", fg="red")
            self.parent.acn.label_acn_device_name_value.configure(text="")
            self.parent.acn.label_acn_device_hid_value.configure(text="")
        elif response == Response.SAFARI_FAULT_DETECTED:
            if DEBUG:
                print("Fault Detected!")
            msg.showerror("Safari Fault", "Accelerometer Fault Condition Detected.")
        elif response == Response.SAFARI_FAULT_CLEARED:
            if DEBUG:
                print("Fault Cleared!")

    # Function for initializing the MQTT connection with Arrow Connect
    def mqtt_init(self):
        meerkat.acn.mqtt_init(self.mqtt_callback)


In [ ]:
class Motor(tk.Frame):
    def __init__(self, parent, master, tab):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.master = master

        self.frame = tk.LabelFrame(tab, height=100, bd=3, relief='groove', text="Motor", fg="blue")
        self.frame.pack(padx=7, pady=7, fill=tk.BOTH)

        self.btn_start_motor = tk.Button(self.frame, text="Start", command=self.motor_start)
        self.btn_start_motor.pack(fill=tk.BOTH, padx=7, pady=7)

    # Function for starting the motor
    def motor_start(self):
        if not meerkat.safari.motor_running:
            if meerkat.serial_port_open():
                meerkat.motor_start()
                self.btn_start_motor.configure(text="Stop")
            else:
                msg.showerror("Serial Port Error", "Serial Port Not Open.")
        else:
            if meerkat.serial_port_open():
                meerkat.motor_stop()
                self.btn_start_motor.configure(text="Start")
            else:
                msg.showerror("Serial Port Error", "Serial Port Not Open.")

    # Function for removing motor controls from the GUI
    def forget(self):
        self.btn_start_motor.pack_forget()
        self.frame.pack_forget()


In [ ]:
class Data(tk.Frame):
    def __init__(self, parent, master, tab):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.master = master
        self.update_interval = 100  # Time (ms) between polling/animation updates
        self.dfont = self.parent.font

        self.frame_data = tk.Frame(tab, bd=3, bg="white", relief='groove')
        self.frame_data.pack(fill=tk.BOTH, expand=1)

        self.btn_start_safari = tk.Button(self.frame_data,
                                          text="Start Demo",
                                          fg="white",
                                          bg="green",
                                          command=self.safari_start)
        self.btn_start_safari.grid(row=0, column=4, padx=7, pady=7, sticky=tk.W + tk.E)

        # Create figure for plotting data
        self.fig_data = figure.Figure(figsize=(10, 6))
        self.ax1 = self.fig_data.add_subplot(3, 1, 1)
        self.ax2 = self.fig_data.add_subplot(3, 1, 2)
        self.ax3 = self.fig_data.add_subplot(3, 1, 3)
        self.fig_data.subplots_adjust(hspace=0.5, wspace=0.6, top=0.95, bottom=0.05)

        color = 'tab:blue'
        self.ax1.set_title("Channel 2", fontsize=12)
        self.ax1.set_ylabel('Voltage (V)', color=color)
        self.ax1.tick_params(axis='y', labelcolor=color)

        color = 'tab:red'
        self.ax2.set_title("Channel 4")
        self.ax2.set_ylabel('Voltage (V)', color=color)
        self.ax2.tick_params(axis='y', labelcolor=color)

        color = 'tab:green'
        self.ax3.set_title("Channel 6")
        self.ax3.set_ylabel('Voltage (V)', color=color)
        self.ax3.tick_params(axis='y', labelcolor=color)

        self.fargs_data = (self.ax1, self.ax2, self.ax3)

        # Create a Tk Canvas widget out of our figures
        self.canvas_data = FigureCanvasTkAgg(self.fig_data, master=self.frame_data)
        self.canvas_data_plot = self.canvas_data.get_tk_widget()

        self.data_plot_format = tk.IntVar()
        btn_raw = tk.Radiobutton(self.frame_data,
                                 text="Raw",
                                 variable=self.data_plot_format,
                                 font=self.dfont,
                                 bg="white",
                                 highlightthickness=0,
                                 value=1)
        btn_converted = tk.Radiobutton(self.frame_data,
                                       text="Converted",
                                       variable=self.data_plot_format,
                                       font=self.dfont,
                                       bg="white",
                                       highlightthickness=0,
                                       value=2)
        self.data_plot_format.set(2)

        # Lay out widgets in a grid in the frame
        tk.Label(self.frame_data, text="Data Streaming Method: ", font=self.dfont, bg="white").grid(row=0, sticky=tk.E)
        tk.Label(self.frame_data, text="None", font=self.dfont, bg="white", fg="red").grid(row=0, column=1, sticky=tk.W)
        self.canvas_data_plot.grid(row=1,
                                   column=0,
                                   rowspan=6,
                                   columnspan=5,
                                   sticky=tk.W + tk.E + tk.N + tk.S)

        tk.Label(self.frame_data, text="Data Format:", font=self.dfont, bg="white").grid(row=8, sticky=tk.E + tk.W)
        btn_raw.grid(row=8, column=1, pady=10)
        btn_converted.grid(row=8, column=2, pady=10)

        # Add a standard 5 pixel padding to all widgets
        for w in self.frame_data.winfo_children():
            w.grid(padx=5, pady=5)

        # Make it so that the grid cells expand out to fill window
        for i in range(0, 5):
            self.frame_data.rowconfigure(i, weight=1)
        for i in range(0, 5):
            self.frame_data.columnconfigure(i, weight=1)

        self.annot_ax1 = self.ax1.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                           bbox=dict(boxstyle="round", fc="w"),
                                           arrowprops=dict(arrowstyle="->"))
        self.annot_ax2 = self.ax2.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                           bbox=dict(boxstyle="round", fc="w"),
                                           arrowprops=dict(arrowstyle="->"))
        self.annot_ax3 = self.ax3.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                           bbox=dict(boxstyle="round", fc="w"),
                                           arrowprops=dict(arrowstyle="->"))

        self.annot_ax1_visible = False
        self.annot_ax2_visible = False
        self.annot_ax3_visible = False

        self.annot_ax1_text = ""
        self.annot_ax2_text = ""
        self.annot_ax3_text = ""

        self.line_ax1 = None
        self.line_ax2 = None
        self.line_ax3 = None
        self.annot_ax1_xy = None
        self.annot_ax2_xy = None
        self.annot_ax3_xy = None

        # https://bit.ly/2AWOqiu
        self.fig_data.canvas.mpl_connect("motion_notify_event", self.hover)

        # Call animate() function periodically
        self.ani_data = animation.FuncAnimation(self.fig_data,
                                                self.animate_data,
                                                fargs=self.fargs_data,
                                                interval=self.update_interval)

    # Function for starting the demo on the device
    def safari_start(self):
        if not meerkat.safari.running:
            if meerkat.serial_port_open():
                meerkat.safari.reset()
                meerkat.motor_start()
                meerkat.safari_start()
                self.btn_start_safari.configure(text="Stop Demo", fg="black", bg="red")
            else:
                msg.showerror("Serial Port Error", "Serial Port Not Open.")
        else:
            meerkat.safari_stop()
            meerkat.motor_stop()
            self.btn_start_safari.configure(text="Start Demo", fg="white", bg="green")
            tk.Label(self.frame_data,
                     text="None", font=self.dfont, bg="white", fg="red").grid(row=0, column=1, sticky=tk.W)

    # https://bit.ly/2AWOqiu
    # Function for updating the hovering annotations
    def update_annot(self, axis, ind):
        if axis == self.ax1:
            x, y = self.line_ax1.get_data()
            self.annot_ax1_xy = (x[ind["ind"][0]], y[ind["ind"][0]])
            self.annot_ax1_text = "{}".format(y[ind["ind"][0]])
        elif axis == self.ax2:
            x, y = self.line_ax2.get_data()
            self.annot_ax2_xy = (x[ind["ind"][0]], y[ind["ind"][0]])
            self.annot_ax2_text = "{}".format(y[ind["ind"][0]])
        elif axis == self.ax3:
            x, y = self.line_ax3.get_data()
            self.annot_ax3_xy = (x[ind["ind"][0]], y[ind["ind"][0]])
            self.annot_ax3_text = "{}".format(y[ind["ind"][0]])

    # Callback for the hover event
    def hover(self, event):
        vis_ax1 = self.annot_ax1.get_visible()
        vis_ax2 = self.annot_ax2.get_visible()
        vis_ax3 = self.annot_ax3.get_visible()
        try:
            if event.inaxes == self.ax1:
                if self.line_ax1:
                    cont, ind = self.line_ax1.contains(event)
                    if cont:
                        self.update_annot(self.ax1, ind)
                        self.annot_ax1_visible = True
                    else:
                        if vis_ax1:
                            self.annot_ax1_visible = False

            elif event.inaxes == self.ax2:
                if self.line_ax2:
                    cont, ind = self.line_ax2.contains(event)
                    if cont:
                        self.update_annot(self.ax2, ind)
                        self.annot_ax2_visible = True
                    else:
                        if vis_ax2:
                            self.annot_ax2_visible = False

            if event.inaxes == self.ax3:
                if self.line_ax3:
                    cont, ind = self.line_ax3.contains(event)
                    if cont:
                        self.update_annot(self.ax3, ind)
                        self.annot_ax3_visible = True
                    else:
                        if vis_ax3:
                            self.annot_ax3_visible = False
        except IndexError:
            print("Uh-Oh")

    # This function is called periodically from FuncAnimation
    def animate_data(self, i, ax1, ax2, ax3):
        # Ignored parameters
        del i

        ax1_color = 'tab:blue'
        ax2_color = 'tab:red'
        ax3_color = 'tab:green'
        plot_title_fontsize = 12
        plot_label_fontsize = 12

        if DEBUG:
            print("Animating Data!!!")

        # Only check the dimensions if connected to the cloud since serial data will generate it's own x-axis array
        if meerkat.cloud_connected:
            # Check data array dimensions and flush if they don't match
            if len(meerkat.safari.ch4_timestamps) != len(meerkat.safari.ch4_voltages) or \
                    len(meerkat.safari.ch4_timestamps) != len(meerkat.safari.ch4_values):
                print("Invalid Channel 4 Array Dimensions. Don't try to animate.")
                meerkat.safari.ch4_timestamps.clear()
                meerkat.safari.ch4_voltages.clear()
                meerkat.safari.ch4_values.clear()
                return

            if len(meerkat.safari.ch6_timestamps) != len(meerkat.safari.ch6_voltages) or \
                    len(meerkat.safari.ch6_timestamps) != len(meerkat.safari.ch6_values):
                print("Invalid Channel 6 Array Dimensions. Don't try to animate.")
                meerkat.safari.ch6_timestamps.clear()
                meerkat.safari.ch6_voltages.clear()
                meerkat.safari.ch6_values.clear()
                return

        self.ax1.clear()
        self.ax2.clear()
        self.ax3.clear()
        self.ax1.tick_params(axis='y', labelcolor=ax1_color)
        self.ax2.tick_params(axis='y', labelcolor=ax2_color)
        self.ax3.tick_params(axis='y', labelcolor=ax3_color)

        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        # Clear, format, and plot voltage values
        if self.data_plot_format.get() == 1:
            self.ax1.set_title("Channel 0 Voltage", fontsize=plot_title_fontsize)
            self.ax1.set_ylabel('Voltage (V)', color=ax1_color, fontsize=plot_label_fontsize)
            self.ax2.set_title("Channel 4 Voltage", fontsize=plot_title_fontsize)
            self.ax2.set_ylabel('Voltage (mV)', color=ax2_color, fontsize=plot_label_fontsize)
            self.ax3.set_title("Channel 6 Voltage", fontsize=plot_title_fontsize)
            self.ax3.set_ylabel('Voltage (V)', color=ax3_color, fontsize=plot_label_fontsize)
            self.ax1.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
            self.ax2.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))
            self.ax3.yaxis.set_major_formatter(FormatStrFormatter('%.4f'))

            try:
                if len(meerkat.safari.ch0_voltages) > 0:
                    self.ax1.plot(np.arange(len(meerkat.safari.ch0_voltages)), meerkat.safari.ch0_voltages,
                                  linewidth=2, color=ax1_color)
                    self.line_ax1 = self.ax1.get_lines()[0]
                    self.annot_ax1 = self.ax1.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                       bbox=dict(boxstyle="round", fc="w"),
                                                       arrowprops=dict(arrowstyle="->"))
                    self.annot_ax1.set_visible(self.annot_ax1_visible)
                    self.annot_ax1.xy = self.annot_ax1_xy
                    self.annot_ax1.set_text(self.annot_ax1_text)
                    self.annot_ax1.get_bbox_patch().set_alpha(0.4)
                    try:
                        ax1_min = min(meerkat.safari.ch0_voltages)
                        ax1_mean = sum(meerkat.safari.ch0_voltages) / len(meerkat.safari.ch0_voltages)
                        ax1_max = max(meerkat.safari.ch0_voltages)
                        ax1_last = meerkat.safari.ch0_voltages[-1]
                        ax1_annotation = '\n'.join((
                            r'Max=%.4f' % (ax1_max,),
                            r'Mean=%.4f' % (ax1_mean,),
                            r'Min=%.4f' % (ax1_min,),
                            r'Last=%.4f' % (ax1_last,)))

                        self.ax1.text(1.01, 0.7, ax1_annotation, transform=self.ax1.transAxes, fontsize=10,
                                      verticalalignment='top', bbox=props)
                    except ValueError:
                        print("Dimension Error")

                if len(meerkat.safari.ch4_voltages) > 0:
                    if meerkat.cloud_connected:
                        self.ax2.plot(meerkat.safari.ch4_timestamps, meerkat.safari.ch4_voltages,
                                      linewidth=2, color=ax2_color)
                    else:
                        self.ax2.plot(np.arange(len(meerkat.safari.ch4_voltages)), meerkat.safari.ch4_voltages,
                                      linewidth=2, color=ax2_color)
                    self.line_ax2 = self.ax2.get_lines()[0]
                    self.annot_ax2 = self.ax2.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                       bbox=dict(boxstyle="round", fc="w"),
                                                       arrowprops=dict(arrowstyle="->"))
                    self.annot_ax2.set_visible(self.annot_ax2_visible)
                    self.annot_ax2.xy = self.annot_ax2_xy
                    self.annot_ax2.set_text(self.annot_ax2_text)
                    self.annot_ax2.get_bbox_patch().set_alpha(0.4)
                    try:
                        ax2_min = min(meerkat.safari.ch4_voltages)
                        ax2_mean = sum(meerkat.safari.ch4_voltages) / len(meerkat.safari.ch4_voltages)
                        ax2_max = max(meerkat.safari.ch4_voltages)
                        ax2_last = meerkat.safari.ch4_voltages[-1]
                        ax2_annotation = '\n'.join((
                            r'Max=%.4f' % (ax2_max,),
                            r'Mean=%.4f' % (ax2_mean,),
                            r'Min=%.4f' % (ax2_min,),
                            r'Last=%.4f' % (ax2_last,)))
                        self.ax2.text(1.01, 0.7, ax2_annotation, transform=self.ax2.transAxes, fontsize=10,
                                      verticalalignment='top', bbox=props)
                    except ValueError:
                        print("Dimension Error")

                if len(meerkat.safari.ch6_voltages) > 0:
                    if meerkat.cloud_connected:
                        self.ax3.plot(meerkat.safari.ch6_timestamps, meerkat.safari.ch6_voltages,
                                      linewidth=2, color=ax3_color)
                    else:
                        self.ax3.plot(np.arange(len(meerkat.safari.ch6_voltages)), meerkat.safari.ch6_voltages,
                                      linewidth=2, color=ax3_color)
                    self.line_ax3 = self.ax3.get_lines()[0]
                    self.annot_ax3 = self.ax3.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                       bbox=dict(boxstyle="round", fc="w"),
                                                       arrowprops=dict(arrowstyle="->"))
                    self.annot_ax3.set_visible(self.annot_ax3_visible)
                    self.annot_ax3.xy = self.annot_ax3_xy
                    self.annot_ax3.set_text(self.annot_ax3_text)
                    self.annot_ax3.get_bbox_patch().set_alpha(0.4)
                    try:
                        ax3_min = min(meerkat.safari.ch6_voltages)
                        ax3_mean = sum(meerkat.safari.ch6_voltages) / len(meerkat.safari.ch6_voltages)
                        ax3_max = max(meerkat.safari.ch6_voltages)
                        ax3_last = meerkat.safari.ch6_voltages[-1]
                        ax3_annotation = '\n'.join((
                            r'Max=%.4f' % (ax3_max,),
                            r'Mean=%.4f' % (ax3_mean,),
                            r'Min=%.4f' % (ax3_min,),
                            r'Last=%.4f' % (ax3_last,)))
                        self.ax3.text(1.01, 0.7, ax3_annotation, transform=self.ax3.transAxes, fontsize=10,
                                      verticalalignment='top', bbox=props)
                    except ValueError:
                        print("Dimension Error")

            except Exception as e:
                logging.exception(e)
                print("Dimension Error")

        # Clear, format, and plot the converted voltage values
        elif self.data_plot_format.get() == 2:
            self.ax1.set_title("Accelerometer", fontsize=plot_title_fontsize)
            self.ax1.set_ylabel("Accel. (g)", color=ax1_color, fontsize=plot_label_fontsize)
            self.ax2.set_title("Temperature", fontsize=plot_title_fontsize)
            self.ax2.set_ylabel("Temp. (deg C)", color=ax2_color, fontsize=plot_label_fontsize)
            self.ax3.set_title("Pressure", fontsize=plot_title_fontsize)
            self.ax3.set_ylabel("Pressure (Pa)", color=ax3_color, fontsize=plot_label_fontsize)
            self.ax1.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
            self.ax3.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

            try:
                if len(meerkat.safari.ch0_values) > 0:
                    ax1.plot(np.arange(len(meerkat.safari.ch0_values)), meerkat.safari.ch0_values,
                             linewidth=2, color=ax1_color)
                    self.line_ax1 = self.ax1.get_lines()[0]
                    self.annot_ax1 = self.ax1.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                       bbox=dict(boxstyle="round", fc="w"),
                                                       arrowprops=dict(arrowstyle="->"))
                    self.annot_ax1.set_visible(self.annot_ax1_visible)
                    self.annot_ax1.xy = self.annot_ax1_xy
                    self.annot_ax1.set_text(self.annot_ax1_text)
                    self.annot_ax1.get_bbox_patch().set_alpha(0.4)
                    try:
                        ax1_min = min(meerkat.safari.ch0_values)
                        ax1_mean = sum(meerkat.safari.ch0_values) / len(meerkat.safari.ch0_values)
                        ax1_max = max(meerkat.safari.ch0_values)
                        ax1_last = meerkat.safari.ch0_values[-1]
                        ax1_annotation = '\n'.join((
                            r'Max=%.3f' % (ax1_max,),
                            r'Mean=%.3f' % (ax1_mean,),
                            r'Min=%.3f' % (ax1_min,),
                            r'Last=%.3f' % (ax1_last,)))

                        self.ax1.text(1.01, 0.7, ax1_annotation, transform=self.ax1.transAxes, fontsize=10,
                                      verticalalignment='top', bbox=props)
                    except ValueError:
                        print("Dimension Error")

                if len(meerkat.safari.ch4_values) > 0:
                    if meerkat.cloud_connected:
                        ax2.plot(meerkat.safari.ch4_timestamps, meerkat.safari.ch4_values,
                                 linewidth=2, color=ax2_color)
                    else:
                        ax2.plot(np.arange(len(meerkat.safari.ch4_values)), meerkat.safari.ch4_values,
                                 linewidth=2, color=ax2_color)
                    self.line_ax2 = self.ax2.get_lines()[0]
                    self.annot_ax2 = self.ax2.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                       bbox=dict(boxstyle="round", fc="w"),
                                                       arrowprops=dict(arrowstyle="->"))
                    self.annot_ax2.set_visible(self.annot_ax2_visible)
                    self.annot_ax2.xy = self.annot_ax2_xy
                    self.annot_ax2.set_text(self.annot_ax2_text)
                    self.annot_ax2.get_bbox_patch().set_alpha(0.4)
                    try:
                        ax2_min = min(meerkat.safari.ch4_values)
                        ax2_mean = sum(meerkat.safari.ch4_values) / len(meerkat.safari.ch4_values)
                        ax2_max = max(meerkat.safari.ch4_values)
                        ax2_last = meerkat.safari.ch4_values[-1]
                        ax2_annotation = '\n'.join((
                            r'Max=%.2f' % (ax2_max,),
                            r'Mean=%.2f' % (ax2_mean,),
                            r'Min=%.2f' % (ax2_min,),
                            r'Last=%.2f' % (ax2_last,)))

                        self.ax2.text(1.01, 0.7, ax2_annotation, transform=self.ax2.transAxes, fontsize=10,
                                      verticalalignment='top', bbox=props)
                    except ValueError:
                        print("Dimension Error")

                if len(meerkat.safari.ch6_values) > 0:
                    if meerkat.cloud_connected:
                        ax3.plot(meerkat.safari.ch6_timestamps, meerkat.safari.ch6_values,
                                 linewidth=2, color=ax3_color)
                    else:
                        ax3.plot(np.arange(len(meerkat.safari.ch6_values)), meerkat.safari.ch6_values,
                                 linewidth=2, color=ax3_color)
                    self.line_ax3 = self.ax3.get_lines()[0]
                    self.annot_ax3 = self.ax3.annotate("", xy=(0, 0), xytext=(-20, 20), textcoords="offset points",
                                                       bbox=dict(boxstyle="round", fc="w"),
                                                       arrowprops=dict(arrowstyle="->"))
                    self.annot_ax3.set_visible(self.annot_ax3_visible)
                    self.annot_ax3.xy = self.annot_ax3_xy
                    self.annot_ax3.set_text(self.annot_ax3_text)
                    self.annot_ax3.get_bbox_patch().set_alpha(0.4)
                    try:
                        ax3_min = min(meerkat.safari.ch6_values)
                        ax3_mean = sum(meerkat.safari.ch6_values) / len(meerkat.safari.ch6_values)
                        ax3_max = max(meerkat.safari.ch6_values)
                        ax3_last = meerkat.safari.ch6_values[-1]
                        ax3_annotation = '\n'.join((
                            r'Max=%.2f' % (ax3_max,),
                            r'Mean=%.2f' % (ax3_mean,),
                            r'Min=%.2f' % (ax3_min,),
                            r'Last=%.2f' % (ax3_last,)))

                        self.ax3.text(1.01, 0.7, ax3_annotation, transform=self.ax3.transAxes, fontsize=10,
                                      verticalalignment='top', bbox=props)
                    except ValueError:
                        print("Dimension Error")
            except Exception as e:
                logging.exception(e)
                print("Dimension Error")

        if len(meerkat.safari.ch4_timestamps) > 0:
            try:
                ax2.xaxis.set_major_formatter(mdates.DateFormatter('%M:%S.%f'))
            except Exception as e:
                logging.exception(e)
                pass

        if len(meerkat.safari.ch6_timestamps) > 0:
            try:
                ax3.xaxis.set_major_formatter(mdates.DateFormatter('%M:%S.%f'))
            except Exception as e:
                logging.exception(e)
                pass


In [ ]:
class Terminal(tk.Frame):
    def __init__(self, parent, master, tab):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.master = master

        self.frame_terminal = tk.LabelFrame(tab, height=150, bd=3, relief='groove', text="Terminal", fg="blue")
        self.frame_terminal.pack(padx=7, pady=7, fill=tk.BOTH)

        self.frame_command = tk.LabelFrame(tab, height=50, bd=3, relief='groove', text="Command", fg="blue")
        self.frame_command.pack(padx=7, pady=7, fill=tk.BOTH)

        self.frame_spc_terminal = tk.LabelFrame(
            tab, height=50, bd=3, relief='groove', text="Special Characters", fg="blue")
        self.frame_spc_terminal.pack(padx=7, pady=7, fill=tk.BOTH)

        self.log = scrolledtext.ScrolledText(self.frame_terminal, wrap=tk.WORD)
        self.log.pack(fill=tk.BOTH)

        self.command_entry = tk.Entry(self.frame_command)
        self.command_entry.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        self.command_entry.bind("<Return>", self.return_key)
        self.btn_send = tk.Button(self.frame_command, text="Send", command=self.send_serial)
        self.btn_send.pack(side=tk.LEFT)
        self.btn_ctrl_c = tk.Button(self.frame_spc_terminal, text="Ctrl-C", command=meerkat.serial_ctrl_c)
        self.btn_ctrl_c.pack(padx=7, pady=7, side=tk.LEFT)

    def send_serial(self):
        # This function is for sending data from the computer to the host controller.
        #
        # The value entered in the the entry box is pushed to the UART. The data can be of any format, since the
        # data is always converted into UTF-8, the receiving device has to convert the data into the required format.

        # Check if the serial port is open before doing any serial operations.
        if meerkat.serial_port_open():
            # Grab the entry from the command box and append a newline to simulate a carriage return on the host.
            send_data = self.command_entry.get()
            send_data = send_data + "\n"

            if DEBUG:
                print("Sending {}".format(send_data))

            meerkat.serial.write(send_data.encode('utf-8'))

            # Clear the entry box after sending the command
            self.command_entry.delete(0, 'end')
        else:
            msg.showerror("Serial Port Error", "Serial Port Not Open.")

    def return_key(self, event=None):
        # Ignored parameters
        del event
        self.send_serial()



In [ ]:
class Main(tk.Frame):
    def __init__(self, parent, master):
        tk.Frame.__init__(self, parent)
        self.parent = parent
        self.master = master
        self.developer_mode_enabled = False
        # Create dynamic font for text
        self.font = tkfont.Font(size=-12)

        # Tabs
        self.tabs = Tabs(self, self.master)
        self.tab_setup = self.tabs.add_tab("Setup")
        self.tab_data = self.tabs.add_tab("Safari Data")
        self.tab_test = None
        self.tab_terminal = None

        # Frames
        self.instructions = Instructions(self, parent, self.tab_setup)
        self.serial = Serial(self, parent, self.tab_setup)
        # self.wifi = Wifi(self, parent, self.tab_setup)
        self.wifi = None
        # self.acn = Acn(self, parent, self.tab_setup)
        self.acn = None
        self.data = Data(self, parent, self.tab_data)
        self.motor = None
        self.test = None
        self.terminal = None

    def toggle_developer_mode(self):
        self.developer_mode_enabled = not self.developer_mode_enabled

        if self.developer_mode_enabled:
            if DEBUG:
                print("Developer Mode Enabled")
            self.tab_test = self.tabs.add_tab("Test Data")
            self.tab_terminal = self.tabs.add_tab("Terminal")

            self.motor = Motor(self, self.parent, self.tab_setup)
            self.test = SafariTest(self, self.parent, self.tab_test)
            self.terminal = Terminal(self, self.parent, self.tab_terminal)
        else:
            if DEBUG:
                print("Developer Mode Disabled")
            # Remove tabs from the GUI
            self.tabs.remove_tab(self.tab_terminal)
            self.tabs.remove_tab(self.tab_test)
            self.motor.forget()

    def device_setup(self):
        if DEBUG:
            print("Setting Up Device")
        meerkat.setup()

In [ ]:
class MainApplication(tk.Frame):
    def __init__(self, parent, *args, **kwargs):
        tk.Frame.__init__(self, parent, *args, **kwargs)
        self.parent = parent
        self.parent.title("Safari")
        self.state = False

        # Fit contents to window
        self.parent.columnconfigure(0, weight=1)
        self.parent.rowconfigure(0, weight=1)
        self.main = Main(self, parent)
        self.toolbar = Toolbar(self, parent)

        # Call empty _destroy function on exit to prevent segmentation fault
        self.parent.bind("<Destroy>", self._destroy)
        self.parent.bind("<Escape>", self.end_fullscreen)
        self.parent.bind("<F11>", self.toggle_fullscreen)

        # Maximize window
        self.parent.state("zoomed")

    # Dummy function prevents segfault
    def _destroy(self, event):
        pass

    def toggle_fullscreen(self, event=None):
        # ignored parameters
        del event

        self.state = not self.state
        self.parent.attributes("-fullscreen", self.state)

    def end_fullscreen(self, event=None):
        # ignored parameters
        del event

        self.state = False
        self.parent.attributes("-fullscreen", False)
        return "break"

In [ ]:
# Main script
if __name__ == "__main__":
    root = tk.Tk()
    MainApplication(root)
    # mainloop
    root.mainloop()